# Making a Chatbot

## Intro

So, I have always been interested in AI, NLP, and Chatbots, so I tried to make a basic chatbot using the an available intents file from <a href = "https://www.kaggle.com/datasets/elvinagammed/chatbots-intent-recognition-dataset?select=Intent.json"> kaggle. </a>
<p> For this project I used a lot of online tutorials so some code might seem familiar </p>

### Importing Libraries

The libraries we need for this are random to generate random responses; json, to read the intents.json file; pickle to store the model and later call it; nltk, or the natural language toolkit; and tensor flow's Keras to add neural network layers to the model.

In [4]:
import random
import json
import pickle
import numpy as np
import nltk
#!pip install tensorflow
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

  Using cached tensorflow-2.9.1-cp39-cp39-win_amd64.whl (444.0 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached libclang-14.0.1-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
  Using cached tensorflow_io_gcs_filesystem-0.26.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached absl_py-1.1.0-py3-none-any.whl (123 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.wh

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dushy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dushy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dushy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


From NLTK we importer the WordNetLemmatizer. IN NLP we often hear stemmer and lemmatizer a lot which basically are used to reduce the length of the word. However, they both achieve it in different ways.
<p> While stemmer usually removes the suffixes of a word to relate them, lemmatizer actually goes to the core word, or so I have read </p>
<p> For example, [study, studies, studying] under a stemmer would become [study, stud, study] as it removes the suffixer. A lemmatizer will extract the core word/the lemma from these and therefore what we see would be [study, study, study.]
    <p>In theory, lemmatizers are usually better for bigger applications of NLP as they can understand the context of the word...in theory. They can tell the difference between care and caring while a stemmer may store the latter as only "car" and not "the gerund of care"/"the act of taking care of someone."</p> </p>

### Loading the json file and creating empty lists to store words, tags, documents

In [17]:
lemmatizer = WordNetLemmatizer()
#with open ("./intents.json") as d:
#    intents = json.loads(d).read()
intents = json.loads(open('./intents.json').read())

In [18]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',',':']

### Reading the intents file and storing the conversation tags and texts

In [19]:
for intent in intents['intents']:
    for pattern in intent['texts']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag,output_row])

What the last for loop does is it takes the document we have created with the tag of words and with all the individual words from the texts and codes them as 1 or 0 based on whether the word appears in that specific tag or not.
<p> So the document looks something like </p>
<p> ---------------------</p>
<p> "Tag" "Hello" "Bye" "I" "am" "good"</p>
<p> Greeting   | 1 | 0 | 0 | 0 | 0 | </p>
<p> Farewell   | 0 | 1 | 0 | 0 | 0 | </p>
<p> General    | 0 | 0 | 1 | 1 | 1 | </p>
<p> ---------------------</p>

<p> This helps our machine know what goes where because it cannot read words, only binary 1s and 0s. Fun fact: this is also called one-hot encoding sometimes - setting a certain character to 1 when it should be true and 0 when it should be false and not occur.

### Training the model and storing it

Now we get to training our model and saving it so we can call on it later. To train the model we will be using a FeetForward Neural Network, more on that later.
So we will add layers to our model, and the last layer is a softmax layer which is basically providing a probability that our input belongs to a certain tag and then it picks the tag that has the highest probability.
For this instance we want our model to be highly accurate so we will have the metric set to accuracy, then we pass it the training x variables and y variables and we are done!

In [20]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

Flynn_use = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('FlynnBot_Model.h5', Flynn_use)
print('Done')

Epoch 1/200


C:\Users\dushy\AppData\Local\Temp\ipykernel_16800\3965853973.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
Z:\Anaconda\Install\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


28/28 [==============================] - 0s 1ms/step - loss: 3.0830 - accuracy: 0.0292 
Epoch 2/200
28/28 [==============================] - 0s 893us/step - loss: 2.9717 - accuracy: 0.1168
Epoch 3/200
28/28 [==============================] - 0s 1ms/step - loss: 2.8698 - accuracy: 0.1971
Epoch 4/200
28/28 [==============================] - 0s 1ms/step - loss: 2.7628 - accuracy: 0.1971
Epoch 5/200
28/28 [==============================] - 0s 993us/step - loss: 2.5566 - accuracy: 0.3285
Epoch 6/200
28/28 [==============================] - 0s 925us/step - loss: 2.3792 - accuracy: 0.3212
Epoch 7/200
28/28 [==============================] - 0s 1ms/step - loss: 2.2589 - accuracy: 0.3796
Epoch 8/200
28/28 [==============================] - 0s 1ms/step - loss: 2.0317 - accuracy: 0.4234
Epoch 9/200
28/28 [==============================] - 0s 1ms/step - loss: 1.8263 - accuracy: 0.4818
Epoch 10/200
28/28 [==============================] - 0s 1ms/step - loss: 1.7075 - accuracy: 0.5255
Epoch 11/200
2

28/28 [==============================] - 0s 947us/step - loss: 0.2417 - accuracy: 0.9197
Epoch 83/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2376 - accuracy: 0.9270
Epoch 84/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2633 - accuracy: 0.9197
Epoch 85/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2753 - accuracy: 0.9051
Epoch 86/200
28/28 [==============================] - 0s 963us/step - loss: 0.2277 - accuracy: 0.9270
Epoch 87/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2539 - accuracy: 0.9197
Epoch 88/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2689 - accuracy: 0.9270
Epoch 89/200
28/28 [==============================] - 0s 972us/step - loss: 0.2100 - accuracy: 0.9416
Epoch 90/200
28/28 [==============================] - 0s 896us/step - loss: 0.2110 - accuracy: 0.9489
Epoch 91/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2306 - accuracy: 0.9270
Epoch

28/28 [==============================] - 0s 1ms/step - loss: 0.2013 - accuracy: 0.9562
Epoch 163/200
28/28 [==============================] - 0s 1ms/step - loss: 0.1503 - accuracy: 0.9562
Epoch 164/200
28/28 [==============================] - 0s 1ms/step - loss: 0.1868 - accuracy: 0.9416
Epoch 165/200
28/28 [==============================] - 0s 889us/step - loss: 0.1665 - accuracy: 0.9416
Epoch 166/200
28/28 [==============================] - 0s 1ms/step - loss: 0.1650 - accuracy: 0.9416
Epoch 167/200
28/28 [==============================] - 0s 889us/step - loss: 0.1094 - accuracy: 0.9708
Epoch 168/200
28/28 [==============================] - 0s 1ms/step - loss: 0.1328 - accuracy: 0.9635
Epoch 169/200
28/28 [==============================] - 0s 963us/step - loss: 0.2035 - accuracy: 0.9489
Epoch 170/200
28/28 [==============================] - 0s 1000us/step - loss: 0.1613 - accuracy: 0.9343
Epoch 171/200
28/28 [==============================] - 0s 1ms/step - loss: 0.1804 - accuracy: 0.

## Running the chatbot

So to run the chatbot we need to define some functions like cleaning up the input sentence. That funtion is handeled by the tokenizer which we have seen above as well. Tokenizer basically breaks down the sentence into different words and characters and then returns a list containing all these tokens.

In [21]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

Flynn = load_model('FlynnBot_Model.h5') ##do not forget to load your model!

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

Now we can define a function to take these tokenised words and convert them into a bag of words with one hot encoding (1s and 0s)

In [22]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

Then we predict what class/tag this sentence, or bag of words, belongs to.

In [23]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = Flynn.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
    return return_list

Then finally a function to use the input and tell our model to get us a response from the appropriate class.

In [24]:
def get_response(intents_list,intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

## Now we can finally run the bot.

I set the initial message to be a random word that I do not think anyone would ever say to the bot so that it can start with the outpit of "hello."

In [25]:
print('GO! Bot is running!')
message = "Shambala"

while message != 'quit':
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)
    message = input("").lower()

GO! Bot is running!
1/1 [==============================] - 0s 66ms/step
Hola, please tell me your name
Hi, my name is Rick
1/1 [==============================] - 0s 14ms/step
Good! Hi <HUMAN>, how can I help you?
Tell me a joke
1/1 [==============================] - 0s 14ms/step
An old man takes his two grandchildren to see the new Scooby-Doo film. When he returns home, his wife asks if he enjoyed himself. 'Well', he starts, 'if it wasn't for those pesky kids...!'
Another joke
1/1 [==============================] - 0s 13ms/step
A doctor thoroughly examined his patient and said, 'Look I really can't find any reason for this mysterious affliction. It's probably due to drinking.' The patient sighed and snapped, 'In that case, I'll come back when you're damn well sober!'
Okay, give me some gossip
1/1 [==============================] - 0s 13ms/step
Kathy said he sense that I are trying to prevent him from closing this conversation why is that.
I don't know
1/1 [=============================

# TA DA!